In [1]:
# Run in terminal or command prompt
# python3 -m spacy download en
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools
# import pyLDAvis
# import pyLDAvis.sklearn
# import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\Yassine\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Yassine\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://num

In [2]:
df = pd.read_csv("ScoutIT_datasets.csv", error_bad_lines=False)
df = df.dropna(subset=["Resume"])

In [3]:
# Convert to list
data = df.Resume.values.tolist()
# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
# pprint(data[:1])

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
# print(data_words[:1])

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [6]:
# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nlp = spacy.load("en", disable=["parser", "ner"])
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN", "VERB", "ADJ", "ADVERB"]) #select noun and verb
print(data_lemmatized[:2])

['technical skill high secondary engineer sslc finance detail human resource finance exprience year monthscompany detail company description', 'skill word look pivot table other basic function power point saral payment package payroll software internet detail bachelor hospitality detail management exprience less year monthscompany detail company atri developer description performance management company description employee relation administration create industry specific policy procedure form format letter checklist payroll management salary restructuring process payroll employee validation input attendance leave salary start salary process process accurate error free salary employee responsible compensation benefit coordinate account team salary processing attendance leave record management assure prompt satisfactory resolution payroll relate query employee statutory compliance management manage various statutory compliance requirement esic gratuity tds calculation deduction payment r

In [8]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=1,
                             max_df= 50,
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                            )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [9]:
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,               
                                      learning_method='online',   
                                      random_state=100,          
                                      batch_size=128,            
                                      evaluate_every = -1,       
                                      n_jobs = -1,               
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [10]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
 evaluate_every=-1, learning_decay=0.7,
 learning_method="online", learning_offset=10.0,
 max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
 n_components=10, n_jobs=-1, perp_tol=0.1,
 random_state=100, topic_word_prior=None,
 total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [11]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -89340.24849932142
Perplexity:  1203.209174852408
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [12]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
            perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0), iid=True, n_jobs=1,
       param_grid={'n_topics': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

C:\Users\Yassine\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Yassine\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
C:\Users\Yassine\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarn

GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method=None,
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=1,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 total_

In [13]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -46620.14610880194
Model Perplexity:  2544.210909332303


In [14]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Category" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Job" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic["dominant_category"] = dominant_topic
# Styling
def color_green(val):
 color = "green" if val > .05 else "black"
 return "color: {col}".format(col=color)
def make_bold(val):
 weight = 700 if val > .05 else 400
 return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topics

,Category0,Category1,Category2,Category3,Category4,Category5,Category6,Category7,Category8,Category9,dominant_category
Job0,0.010000,0.010000,0.010000,0.930000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,3
Job1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,9
Job2,0.030000,0.030000,0.030000,0.770000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,3
Job3,0.030000,0.030000,0.030000,0.030000,0.700000,0.030000,0.030000,0.030000,0.030000,0.030000,4
Job4,0.030000,0.030000,0.030000,0.030000,0.700000,0.030000,0.030000,0.030000,0.030000,0.030000,4
Job5,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,3
Job6,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,7
Job7,0.000000,0.000000,0.000000,0.980000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
Job8,0.010000,0.010000,0.010000,0.940000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,3
Job9,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900000,9


In [15]:
# Topic-Keyword Matrix
df_category_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_category_keywords.columns = vectorizer.get_feature_names()
df_category_keywords.index = topicnames
# View
df_category_keywords

,ability,able,academic,academy,accenture,accept,acceptance,access,accessory,accommodate,...,workforce,working,workshop,world,write,writer,writing,www,year,young
Category0,1.055454,0.547569,0.624766,0.483142,0.523329,0.548081,0.565448,0.534058,0.549711,1.589151,...,0.526569,0.437179,0.780652,0.728130,0.584630,0.578490,0.516370,0.448469,0.593623,0.573335
Category1,0.539956,0.541110,0.507893,0.550313,0.531627,0.505089,0.613090,0.589874,0.530618,0.587422,...,0.537962,0.537539,0.516595,0.549615,0.528293,0.533812,0.476055,0.576112,0.716456,0.653889
Category2,0.709004,0.543519,0.580693,0.602920,0.484420,0.530221,0.511051,0.450420,0.499656,0.503253,...,0.484455,0.552772,0.621662,0.505712,0.782876,0.533976,0.506715,0.562833,0.732584,0.524287
Category3,1.465080,2.433024,1.602623,0.576702,1.403806,0.554242,0.591613,1.522974,0.489601,0.519806,...,0.656509,2.420979,5.400469,0.553440,3.901635,1.512078,0.616042,3.513314,45.104485,1.460060
Category4,0.594183,0.515123,0.520911,0.529237,0.541370,0.477508,0.494210,0.599895,0.450737,0.519802,...,0.463060,0.512613,0.589582,0.505948,0.501207,0.514462,0.530969,0.685260,0.951716,0.673998
Category5,0.560010,0.515870,0.524009,0.477311,0.530394,0.453957,0.477747,0.484609,0.714589,0.443531,...,0.663092,0.474053,0.544277,0.559302,1.617304,0.493683,0.951294,0.575217,1.045010,0.572043
Category6,0.516563,0.519083,0.517827,0.546721,0.639196,0.553449,0.478379,0.502251,0.529369,0.521322,...,0.547449,0.605960,3.500318,1.475938,0.576479,0.561093,0.500360,0.510627,0.976751,0.548956
Category7,2.073227,0.501305,0.480430,1.621699,0.585489,2.055644,0.980183,0.518258,0.970851,0.528034,...,0.617267,0.523898,0.754657,1.268526,1.872462,0.626004,0.532963,0.533403,1.622107,0.461934
Category8,0.567244,0.548383,0.480103,0.557629,0.526508,0.508368,0.579010,0.527558,0.544591,0.538561,...,0.471971,0.572203,0.473918,0.623035,0.449187,0.507763,0.516516,0.536691,0.444218,0.505531
Category9,1.115757,1.315511,0.646430,0.563620,0.461630,1.094906,0.598179,0.569515,0.602783,0.469523,...,0.916951,0.675262,2.624662,0.749374,1.602742,0.618537,1.080118,0.620113,7.928446,0.577496


In [16]:
# Show top n keywords for each topic
def show_category(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    category_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        category_keywords.append(keywords.take(top_keyword_locs))
    return category_keywords
category_keywords = show_category(vectorizer=vectorizer, lda_model=best_lda_model, n_words=50)
# Topic - Keywords Dataframe
df_category_keywords = pd.DataFrame(category_keywords)
df_category_keywords.columns = ['Word '+str(i) for i in range(df_category_keywords.shape[1])]
df_category_keywords.index = ['Category '+str(i) for i in range(df_category_keywords.shape[0])]
pd.set_option('display.max_columns', None)
df_category_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19,Word 20,Word 21,Word 22,Word 23,Word 24,Word 25,Word 26,Word 27,Word 28,Word 29,Word 30,Word 31,Word 32,Word 33,Word 34,Word 35,Word 36,Word 37,Word 38,Word 39,Word 40,Word 41,Word 42,Word 43,Word 44,Word 45,Word 46,Word 47,Word 48,Word 49
Category 0,nutrition,high,project,fitness,goal,civil,condition,group,level,provide,court,achieve,vary,report,organize,present,inspection,turn,site,counselling,matter,consumer,assist,paper,player,forum,follow,detailed,participant,background,drawing,coordinate,obesity,manager,age,telephonic,weight,hypertension,cultural,indoor,accommodate,health,patient,energy,dental,cycling,metabolic,exercise,accompany,quality
Category 1,customer,training,fitness,college,testing,test,month,window,complete,internal,management,day,exprience,total,prepare,report,role,time,quality,sale,bug,nutrition,business,duration,provide,base,track,generate,manager,development,leader,maintenance,health,requirement,execute,software,supply,developer,hospital,communicate,year,service,stress,transformer,operate,attendance,good,reporting,ensure,pvt
Category 2,paint,art,college,experience,job,class,sale,lecturer,fine,shubhankan,health,knowledge,complete,field,operate,indore,window,faculty,monthscompany,summer,area,search,comfort,artist,physical,draw,disability,improve,place,rangoli,month,marketing,quick,exprience,llb,staff,vidishtra,law,manage,requirement,generate,develop,manager,fitness,business,performance,set,good,client,time
Category 3,month,project,exprience,sale,year,application,use,complete,design,develop,web,developer,software,training,time,manage,role,development,technology,management,requirement,responsibility,website,base,pvt,manager,art,event,test,program,computer,provide,make,college,target,day,marketing,client,experience,programme,technical,good,business,knowledge,internal,report,music,information,various,require
Category 4,salary,solution,contract,economics,art,win,medal,martial,trainer,project,designer,job,net,management,web,ssc,gold,movie,listen,watch,personal,surf,base,month,developer,develop,exprience,fitness,good,represent,player,championship,entermediate,information,design,belt,new,year,music,maintain,time,engineer,customer,cost,programme,cricket,development,play,sport,dbms
Category 5,client,health,aza,legal,public,merchant,college,leadership,nutritionist,lead,presentation,suggest,science,nature,property,individual,meet,speaking,visit,environment,activity,test,net,loan,promoter,develop,ensure,requirement,high,family,experience,sale,role,generating,report,communicate,session,residential,district,manager,deal,partner,communication,plan,notice,standard,law,new,site,provide
Category 6,project,material,vendor,product,customer,site,technical,drawing,cost,civil,time,maintain,engineer,ensure,specification,consultant,coordinate,sale,process,requirement,require,development,engineering,supply,quotation,inspection,order,contractor,design,solution,manufacturing,role,mechanical,preparation,develop,proposal,review,activity,department,quality,client,new,contract,field,progress,support,request,record,follow,document
Category 7,test,aza,testing,duty,inspection,quality,conduct,check,related,project,perform,engineering,transformer,engineer,query,job,customer,complete,handle,assign,month,fitness,people,site,management,handling,therapy,gym,achieve,product,good,electrical,entire,floor,care,goal,pvt,control,refresher,train,relate,report,leader,plan,high,core,instruction,maintain,electronic,construction
Category 8,exprience,mortgage,engineering,aza,platform,visit,financial,lead,residential,labor,enhance,computer,client,merchant,certify,performance,art,suggest,incorporate,notepad,social,compelling,user,easy,production,site,programming,way,commissioning,challenging,month,case,new,hypertension,exception,loan,rota,facilitate,pas,supervise,creative,viable,defence,mistake,technical,map,listen,t

In [22]:
Categories = ["Health and fitness",
              "Testing", 
              "Arts",
              "Web Designer",
              "HR",
              "Advocate",
              "Sales",
              "Python Developer",
              "Mechanical Engineer",                            
              "Civil Engineer"]
df_category_keywords["Categories"]=Categories
df_category_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19,Word 20,Word 21,Word 22,Word 23,Word 24,Word 25,Word 26,Word 27,Word 28,Word 29,Word 30,Word 31,Word 32,Word 33,Word 34,Word 35,Word 36,Word 37,Word 38,Word 39,Word 40,Word 41,Word 42,Word 43,Word 44,Word 45,Word 46,Word 47,Word 48,Word 49,Categories
Category 0,nutrition,high,project,fitness,goal,civil,condition,group,level,provide,court,achieve,vary,report,organize,present,inspection,turn,site,counselling,matter,consumer,assist,paper,player,forum,follow,detailed,participant,background,drawing,coordinate,obesity,manager,age,telephonic,weight,hypertension,cultural,indoor,accommodate,health,patient,energy,dental,cycling,metabolic,exercise,accompany,quality,Health and fitness
Category 1,customer,training,fitness,college,testing,test,month,window,complete,internal,management,day,exprience,total,prepare,report,role,time,quality,sale,bug,nutrition,business,duration,provide,base,track,generate,manager,development,leader,maintenance,health,requirement,execute,software,supply,developer,hospital,communicate,year,service,stress,transformer,operate,attendance,good,reporting,ensure,pvt,Testing
Category 2,paint,art,college,experience,job,class,sale,lecturer,fine,shubhankan,health,knowledge,complete,field,operate,indore,window,faculty,monthscompany,summer,area,search,comfort,artist,physical,draw,disability,improve,place,rangoli,month,marketing,quick,exprience,llb,staff,vidishtra,law,manage,requirement,generate,develop,manager,fitness,business,performance,set,good,client,time,Arts
Category 3,month,project,exprience,sale,year,application,use,complete,design,develop,web,developer,software,training,time,manage,role,development,technology,management,requirement,responsibility,website,base,pvt,manager,art,event,test,program,computer,provide,make,college,target,day,marketing,client,experience,programme,technical,good,business,knowledge,internal,report,music,information,various,require,Web Designer
Category 4,salary,solution,contract,economics,art,win,medal,martial,trainer,project,designer,job,net,management,web,ssc,gold,movie,listen,watch,personal,surf,base,month,developer,develop,exprience,fitness,good,represent,player,championship,entermediate,information,design,belt,new,year,music,maintain,time,engineer,customer,cost,programme,cricket,development,play,sport,dbms,HR
Category 5,client,health,aza,legal,public,merchant,college,leadership,nutritionist,lead,presentation,suggest,science,nature,property,individual,meet,speaking,visit,environment,activity,test,net,loan,promoter,develop,ensure,requirement,high,family,experience,sale,role,generating,report,communicate,session,residential,district,manager,deal,partner,communication,plan,notice,standard,law,new,site,provide,Advocate
Category 6,project,material,vendor,product,customer,site,technical,drawing,cost,civil,time,maintain,engineer,ensure,specification,consultant,coordinate,sale,process,requirement,require,development,engineering,supply,quotation,inspection,order,contractor,design,solution,manufacturing,role,mechanical,preparation,develop,proposal,review,activity,department,quality,client,new,contract,field,progress,support,request,record,follow,document,Sales
Category 7,test,aza,testing,duty,inspection,quality,conduct,check,related,project,perform,engineering,transformer,engineer,query,job,customer,complete,handle,assign,month,fitness,people,site,management,handling,therapy,gym,achieve,product,good,electrical,entire,floor,care,goal,pvt,control,refresher,train,relate,report,leader,plan,high,core,instruction,maintain,electronic,construction,Python Developer
Category 8,exprience,mortgage,engineering,aza,platform,visit,financial,lead,residential,labor,enhance,computer,client,merchant,certify,performance,art,suggest,incorporate,notepad,social,compelling,user,easy,production,site,programming,way,commissioning,challenging,month,case,new,hypertension,exceptio

In [23]:
# Define function to predict category for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])
def predict_category(text, nlp=nlp):
    global sent_to_words
    global lemmatization
# Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))
# Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)
# Step 4: LDA Transform
    category_probability_scores = best_lda_model.transform(mytext_4)
    category = df_category_keywords.iloc[np.argmax(category_probability_scores), 1:14].values.tolist()
    
    # Step 5: Infer category
    infer_category = df_category_keywords.iloc[np.argmax(category_probability_scores), -1]
    
    #category_guess = df_category_keywords.iloc[np.argmax(category_probability_scores), category]
    return infer_category, category, category_probability_scores
# Predict the category
mytext = ["PERSONAL INFORMATION Yassine ELUHARANI Casablanca, 20440 Casablanca (Morocco) +212 644928582 eluharani.yassine@gmail.com EDUCATION AND TRAINING 10/10/2017–Present Bachelor in Business Informatics Université Internationale de Casablanca, Casablanca (Morocco) PERSONAL SKILLS Mother tongue(s) Arabic Foreign language(s) UNDERSTANDING SPEAKING WRITING Listening Reading Spoken interaction Spoken production French C1 C1 B1 B1 B1 English C1 C1 B2 B2 B2 Levels: A1 and A2: Basic user - B1 and B2: Independent user - C1 and C2: Proficient user Common European Framework of Reference for Languages Digital skills SELF-ASSESSMENT Information processing Communication Content creation Safety Problemsolving Independent user Proficient user Proficient user Digital skills - Self-assessment grid Web development : HTML5 , CSS3 , Javascript , PHP , Bootstrap . Mobile development : Android studio , Flutter . Databases : MySQL , SQL server , Microsoft ACCESS , phpmyadmin . Programing language : JAVA , C# , C , Python . Framework : JavaEE , JSP , .NET (ASP , ADO ) . Web servers : Apache Tomcat , Microsoft IIS . Operating systems : Windows , UNIX , LINUX ."]
infer_category, category, prob_scores = predict_category(text = mytext)
print(category)
print(infer_category)

['project', 'exprience', 'sale', 'year', 'application', 'use', 'complete', 'design', 'develop', 'web', 'developer', 'software', 'training']
Web Designer
